In [1]:
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from PyPDF2 import PdfReader
from nlp_rake import Rake
from nltk.corpus import stopwords
from yake import yake

corpus = []
with open('Математическое и информационное моделирование_2020 (1).pdf', 'rb') as f:
    pdf = PdfReader(f)
    for i in range(len(pdf.pages)):
        corpus.append(pdf.pages[i].extract_text())

In [2]:
num_pages = [10, 17, 23, 29, 37, 49, 56, 70, 82, 92, 102, 113, 123, 132, 144, 155, 167, 174, 183, 192, 202, 208, 218,
             226, 235, 242, 251, 260, 266, 277, 288, 299, 307, 314, 326, 335, 345, 352, 359, 366, 377, 385, 397, 404,
             410, 419, 424, 433, 438, 448, 456, 462, 472, 481, 489, 495, 505, 514, 518, 522, 527, 541, 551, 556, 566,
             570, 577, 583, 591, 597, 602, 609, 615, 623, 630, 639, 645, 655, 661, 666, 671, 677, 692, 700, 708, 716,
             726]

In [3]:
stops = list(set(stopwords.words("russian")))
punctuation = set(string.punctuation)

In [4]:
import spacy

nlp = spacy.load("ru_core_news_sm")


def preprocess_and_lemmatize(text):
    doc = nlp(text.lower())
    lemmatized_words = [token.lemma_ for token in doc if token.text not in punctuation and token.text not in stops]
    return " ".join(lemmatized_words)


2023-12-04 12:36:06.404606: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
from nltk import sent_tokenize
sents = [sent_tokenize(i) for i in corpus]

In [6]:
articles = []
for i in range(1, len(num_pages)):
    _ = [corpus[j - 1] for j in range(num_pages[i - 1], num_pages[i])]
    articles.append(_)

In [7]:
article_keywords = []
for a in articles:
    for p in a:
        for s in sent_tokenize(p, 'russian'):
            if 'Ключевые слова' in s:
                article_keywords.append(s)

In [8]:
articles_preprocessed = []
for a in articles:
    pages_preprocessed = [preprocess_and_lemmatize(page) for page in a]
    articles_preprocessed.append([" ".join(pages_preprocessed)])

In [9]:


kw_extractor = yake.KeywordExtractor(lan='ru', n=3)
rake_ = Rake(stopwords=stops, max_words=3)

for idx, (a, a_) in enumerate(zip(articles_preprocessed, articles)):
    vectorizer = TfidfVectorizer(ngram_range=(1, 3))
    tfidf_matrix = vectorizer.fit_transform(a)
    feature_names = vectorizer.get_feature_names_out()

    print('__________________________________________')
    print(article_keywords[idx])

    for scores in tfidf_matrix.toarray():
        sorted_keywords = [word for _, word in sorted(zip(scores, feature_names), reverse=True)][:10]
        print(f"tf-idf: {sorted_keywords}")

    yake_keywords = kw_extractor.extract_keywords(" ".join(a_))
    print(f"Yake: {[i[0] for i in yake_keywords[:10]]}")

    rake_keywords = rake_.apply(" ".join(a_))
    print(f"Rake: { [i[0] for i in sorted(rake_keywords, key=lambda x: x[1], reverse=True)][:10]}")


__________________________________________
Ключевые слова:  математическое  моделирование, специ ализированная 
программа  MDC -SPICE ,  кодирование информации в задержки и в частоту 
импульсов,  виртуальный входной нейрон, популяция нейронов.
tf-idf: ['число', 'схема', 'кодирование', 'входной', 'spice', 'нейрон', 'частота', 'яркость', 'задержка', 'импульс']
Yake: ['SPICE', 'специализированная программа MDC', 'программа MDC', 'КОДИРОВАНИЯ', 'задержки', 'расчета больших электрических', 'числа', 'больших электрических схем', 'яркости', 'яркости пикселей']
Rake: ['входном блоке нейропроцессора', 'значения входного числа', 'биологических нейронных сетях', 'виртуальный входной нейрон', 'первичной обработки звуковых', 'отсев малых амплитуд', 'двоичными чис лами', 'использованием логической матрицы', 'поскольку позволяет реализовать', 'представленной двоичными числами']
__________________________________________
Ключевые слова:  функция Эйлера,  теорема Эйлера, первообразный 
корень, показате